# Script for 2024 Microbiome Workshop

In [ ]:
# Make sure you copy the files to your home directory on sol
# You can use scp or rsync 
# Here is an example: 
# rsync -P *.fastq slwrigh4@login.sol.rc.asu.edu:/home/[your_ASU_ID]

# login into sol with the following: 
# ssh slwrigh4@login.sol.rc.asu.edu

# You can set up an interactive  
# interactive -c 4 --mem=48G

## FastQC

In [ ]:
# Once you are on Sol, you should be able to access the data 
# We will first assess the quality of the data using fastqc 
# Fastqc is installed on Sol
# To access it, we can use the code below
module load fastqc-0.12.1-gcc-11.2.0

# To run fastqc on one sample: 
fastqc SRR23380883-subsample.fastq

# To run fastqc on all fastq files in a directory 
fastqc *.fastqc

## SeqKit

In [ ]:
# Use seqkit to gain more insights about the quality of the data 

# You will need to activate mamba to install seqkit
module load mamba/latest

# Use the following to install seqkit onto Sol
conda install bioconda::seqkit

# Use seqkit stats 
seqkit stats -a *.fastq > seqkit-stats.txt # open in excel 


## Create QIIME2 environment

In [ ]:
# We will need to create a conda environment for the new QIIME2 environment

wget https://data.qiime2.org/distro/amplicon/qiime2-amplicon-2024.5-py39-linux-conda.yml

# You may need to start an interactive session for this to work
# example: interactive -c 4 --mem=48G
conda env create -n qiime2-amplicon-2024.5 --file https://data.qiime2.org/distro/amplicon/qiime2-amplicon-2024.5-py39-linux-conda.yml

# Activate the environment
source activate activate qiime2-amplicon-2024.5

In [ ]:
# Let us import the data into QIIME2 
# Because the following steps will need more RAM, we need to submit batch scripts 
# Copy the entire code below into a text file and save as Import_fastq_to_QIIME2.slurm

## Import data into QIIME2

In [ ]:
#!/bin/bash

#SBATCH --job-name=one-import_fastq_files
#SBATCH -o import_fastq.out
#SBATCH --nodes=1
#SBATCH -t 8:00:00
#SBATCH --cpus-per-task=10
#SBATCH --mem=40G
#SBATCH -p general

cd $SLURM_SUBMIT_DIR

set -uex

# Activate QIIME2 
module load mamba/latest
source activate qiime2-amplicon-2024.5

# Import FASTQ FILES
# You will need to make sure you include the manifest file for these samples
qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path manifest.txt \
  --input-format SingleEndFastqManifestPhred33V2 \
  --output-path pacbio-seqs.qza

# Denoising the sequencing data 

In [ ]:
#!/bin/bash
#SBATCH --job-name=denoise
#SBATCH -o denoise.out
#SBATCH --nodes=1
#SBATCH -t 8:00:00
#SBATCH --cpus-per-task=10
#SBATCH --mem=40G
#SBATCH -p general

cd $SLURM_SUBMIT_DIR

set -uex

# Record the start time
start_time=$(date)

module load mamba/latest

source activate QIIME2-amplicon-2024.5

start_time=$(date)

qiime dada2 denoise-ccs \
--i-demultiplexed-seqs pacbio-seqs.qza \
--p-trim-left 0 \
--p-trunc-len 0 \
--p-front AGRGTTYGATYMTGGCTCAG \
--p-adapter RGYTACCTTGTTACGACTT \
--o-representative-sequences rep-seqs.qza   \
--o-table table.qza   \
--o-denoising-stats stats.qza

end_time=$(date +%s)

runtime=$((end_time - start_time))

echo "Job complete"


## Download Greengenes2 database

In [ ]:
# Download the greengenes2 database 
wget --no-check-certificate https://ftp.microbio.me/greengenes_release/2024.09/2024.09.backbone.full-length.fna.qza

# For the taxonomy file
wget --no-check-certificate https://ftp.microbio.me/greengenes_release/2024.09/2024.09.backbone.tax.qza

## Classify with Vsearch

In [ ]:
#!/bin/bash

#SBATCH --job-name=classify
#SBATCH -o classify.out
#SBATCH --nodes=1
#SBATCH -t 4:00:00
#SBATCH --cpus-per-task=10
#SBATCH --mem=40G
#SBATCH -p general

cd $SLURM_SUBMIT_DIR

set -uex

# Record the start time
start_time=$(date)

module load mamba/latest

source activate QIIME2-amplicon-2024.5

qiime feature-classifier classify-consensus-vsearch \
    --i-query rep-seqs.qza \
    --i-reference-reads 2024.09.backbone.full-length.fna.qza  \
    --i-reference-taxonomy 2024.09.backbone.tax.qza  \
    --p-maxaccepts 1 --p-strand "plus" \
    --p-threads 4 \
    --verbose \
    --output-dir taxa

qiime tools export --input-path taxa/classification.qza --output-path taxa

echo "Job complete"


### Analyses 

In [ ]:
# Create a barplot

qiime taxa barplot \
--i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--m-metadata-file metadata.txt \
--o-visualization taxa-bar-plots.qzv

In [ ]:
# convert taxonomy.tsv file to taxonomy.qza file
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path taxonomy.tsv \
  --output-path taxonomy.qza

# change the header from Feature ID to feature-id
# run the following to check the file
# qiime metadata tabulate --m-input-file taxonomy.qza --o-visualization taxonomy.qzv

In [ ]:
#!/bin/bash

#SBATCH --job-name=denoise
#SBATCH -o denoise.out
#SBATCH --nodes=1
#SBATCH -t 4:00:00
#SBATCH --cpus-per-task=10

cd $SLURM_SUBMIT_DIR

set -uex

# collapse table to the genus level 
qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 6 \
--o-collapsed-table genus-table.qza

qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 6 \
--o-collapsed-table genus-table-2.qza

qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 7 \
--o-collapsed-table species-table.qza

qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 2 \
--o-collapsed-table phylum-table.qza

echo "Job complete"

In [ ]:
# Convert the table from a QZA format to a BIOM or txt format 
# Note that both of these files can be inputted into other programs 

qiime tools export --input-path genus-table.qza --output-path genus-analysis-2
biom convert -i genus-analysis/feature-table.biom -o genus-analysis/feature-table.txt --to-tsv

qiime tools export --input-path species-table.qza --output-path species-analysis
biom convert -i species-analysis/feature-table.biom -o species-analysis/feature-table.txt --to-tsv

In [ ]:
# What if you want to remove singletons and rare taxa that may be inflating diversity estimates and are present due to spurious mappings 

qiime feature-table filter-features \
  --i-table genus-table-2.qza \
  --p-min-samples 2 \
  --p-min-frequency 10 \
  --o-filtered-table filtered-genus-table.qza


## Compositional analyses

In [ ]:
# install gemelli
# make sure your QIIME2 environment is active 
pip install gemelli

In [ ]:
#create a Aitchison distance matrix 

qiime gemelli rpca \
--i-table genus-table.qza \
--o-biplot genus-table-ordination.qza \
--o-distance-matrix genus-table-distance.qza

qiime emperor biplot \
--i-biplot genus-table-ordination.qza \
--m-sample-metadata-file metadata.txt \
--o-visualization genus-biplot.qzv \
--p-number-of-features 1
    
# PERMANOVA results 

NAME=genus
METADATA=metadata.txt
declare -a StringArray=("GroupNumber" "Health")
for category in ${StringArray[@]}; do
qiime diversity beta-group-significance \
  --i-distance-matrix genus-table-distance.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column "$category" \
  --o-visualization ${NAME}-$category-significance.qzv \
  --p-pairwise
done

qiime diversity adonis \
--i-distance-matrix genus-table-distance.qza \
--m-metadata-file $METADATA \
--p-formula "GroupNumber*Health" \
--o-visualization adonis-${NAME}.qzv

## Alpha diversity 

In [ ]:
METRIC=observed_features
qiime diversity alpha \
--i-table genus-table.qza \
--p-metric $METRIC \
--o-alpha-diversity alpha-${NAME}-${METRIC}.qza

qiime diversity alpha-group-significance
--i-alpha-diversity alpha-${NAME}-${METRIC}.qza
--m-metadata-file $METADATA
--o-visualization alpha-${NAME}-${METRIC}.qzv

Additional commands

In [ ]:
# obc2dfatq command

obc2fastq --input <run folder> \
--output <output folder name> \
--flowcellid <flow cell ID> \
--samplesheet <sampleSheet.csv file name> \
--designsheet <obc2fastq_params file name> \
--threadlanes <int> \--threadsperlane <int> \
--controlsfile <control fasta file name> \
--barcodeallowedmismatches <int> \

In [ ]:
# Subsample the fastq files to make them smaller 
# Good when you want to just test whether your program is working without having to do it on the entire dataset

#!/bin/bash

#SBATCH --job-name=denoise
#SBATCH -o denoise.out
#SBATCH --nodes=1
#SBATCH -t 4:00:00
#SBATCH --cpus-per-task=10

cd $SLURM_SUBMIT_DIR

set -uex

# Record the start time
start_time=$(date)

module load mamba/latest

source activate FASTQ_PROCESSING

ls SRR2338099*.fastq | while read samples; do
    base_name=$(basename "$samples" .fastq)  # Extract the base name without .fastq
    seqkit sample -p 0.4 "$samples" -o "${base_name}.4.fastq"  # Use the base name for output
done

# -p is for proportion of sequences to subsample to; here we are collecting 40% of sequences

echo "Job Complete"


In [ ]:
# If you are having trouble with DEICODE, you may need to downgrade your numpy environment
# Make sure your conda environment for QIIME2 is active conda 
mamba install numpy=1.19.5
